Let's do all preprocessing required for fitting the seq-2-seq model here. We need to extract features for the subwindows. We also need to write generic code incase we need to do changes later depending on the performance of the seq-2-seq model.

For the seq-2-seq model, we do not have to touch the densities, and as the first step we should extract the features from each subwindow using the fine-tuned model. We still might need the densities later for the latent AR proces part, we will get there in stage 2. We will need to arange the subwindows in a stack though, so that the features could be extracted. We ahve done this sort of work in AR models, we will look into that too.

In [1]:
import numpy as np
import tensorflow as tf
import os
from skimage.transform import resize

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##### Train and Validation data

Let's first focus on train and validation data. Meaning blocks 101, 102, 203, 301, and 204. We will preprocess data required for seq-2-seq model, we do not have to worry currently about the stage 2 (densities). But how are we going to evaluate the seq-2-seq model? should we do that on the rest of the blocks? Therefore we might have to preprocess the test data too. Let's get block 0101 as the example, develop all functions for this, and work on preprocessing data

For train data, we need to prepare our final input data in the shape None, 13, 32, and the target data by None, 7, 32. For this, we need to use the fin-tuned model to extract the data. To extract 